In [ ]:
#!pip install gradio transformers accelerate peft --quiet

In [ ]:
#!pip install --upgrade transformers --quiet

# Academic Writing LoRa Model: 
<br>
<p>This Project creates an Academic Writing Assistant that generates text continuations for the user in an academic-style.</p>
<p>
The idea for this project started in code, CoPilot has been a massive success that has aided millions of people in coding. Providing real time assistance, as well as an auto-complete function for coding based on previous works of the user. This, significantly improving the experience of users, has not been applied to a wide range in natural language. </p>
<p>
While autocomplete has been around for a while, one area wherein text continuation has not been taken advantage of is in academic writing -- As Copilot has provided a large quality of life experience for those in coding, this project aims to do the same for academic writers. 
</p>

# Why Lora? 
<p>
Lora is a great fit for the initial task of training and fine tuning the model as the original task of the code(text continuation) can already be done on a surface level. 
</p>
<p>
The main adaptation of this code is to create a text continuation for academic writing -- That is, training a model for the specific task of writing for academic writings and scholarly papers. In the context of this code, this means that Lora is perfect for us as it allows us to fine tune an already proven and working base model (AutoModeforCausalM) and create our academic writing auto continuation without the exceedingly high computational costs of training an entire base model by ourselves.  </p>

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel
import gradio as gr

# Load LoRA model using the approach from model_evaluation_and_finetuning.ipynb
model_name = "./lora_academic_model"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model = PeftModel.from_pretrained(base_model, model_name).to(device)
model.eval()

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token



c:\Users\jason\anaconda3\envs\OpenAI\Lib\site-packages\peft\tuners\tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


### TESTING MODEL

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Ensure the fine-tuned model and tokenizer are loaded (from cell AlY-JVt2g7M6)
# If they are not in scope, you might need to run cell AlY-JVt2g7M6 first.

# Define a context for generation
# Using the same context as the stylistic control example
context = "Deep reinforcement learning has been widely adopted in robotic navigation."

# Tokenize the input context
inputs = tokenizer(context, return_tensors="pt").to(device)

# Generate continuation using the fine-tuned model
# You can adjust max_new_tokens, do_sample, temperature, top_p as needed
output_ids = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask, # Explicitly pass attention_mask
    max_new_tokens=100, # Generate up to 100 new tokens
    num_return_sequences=1,
    do_sample=True,      # Enable sampling for more diverse outputs
    temperature=0.7,     # Control creativity (lower for less, higher for more)
    top_p=0.9,           # Nucleus sampling
    pad_token_id=tokenizer.eos_token_id
)

# Decode the generated tokens, skipping the input context and special tokens
generated_text = tokenizer.decode(output_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("Context:", context)
print("\nGenerated Continuation:", generated_text.strip())

Context: Deep reinforcement learning has been widely adopted in robotic navigation.

Generated Continuation: The method can be implemented in a number of different ways, including linear, non-linear, and adaptive. The linear method is widely used in the medical and robotic navigation. The non-linear method is widely used in the medical and robotic navigation.
In the linear method, the position of a robot is determined by the robot's angular velocity, and the position of a robot is determined by the position of a robot's eyes. The position of a robot is a point at which the robot's


### FUNCTION

In [6]:

def generate_continuation(context, max_new_tokens=100, temperature=0.7, top_p=0.9, print_result=True):
    """
    Generate academic text continuation using the loaded LoRA model.
    
    Args:
        context (str): Input prompt/context for generation
        max_new_tokens (int): Maximum number of new tokens to generate (default: 100)
        temperature (float): Sampling temperature - higher for more creativity (default: 0.7)
        top_p (float): Nucleus sampling parameter (default: 0.9)
        print_result (bool): Whether to print the input and output (default: True)
    
    Returns:
        str: Generated continuation text
    """
    # Tokenize the input context
    inputs = tokenizer(context, return_tensors="pt").to(device)
    
    # Generate continuation using the fine-tuned model
    output_ids = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the generated tokens, skipping the input context and special tokens
    generated_text = tokenizer.decode(
        output_ids[0][inputs.input_ids.shape[1]:], 
        skip_special_tokens=True
    )
    
    if print_result:
        print("Context:", context)
        print("\nGenerated Continuation:", generated_text.strip())
        print("-" * 80)
    
    return generated_text.strip()


# Test the function with sample prompts
test_prompts = [
    "Deep reinforcement learning has been widely adopted in robotic navigation.",
    "The key findings of this research are",
    "In recent years, machine learning has demonstrated"
]

print("Testing LoRA model with sample prompts:\n")
for prompt in test_prompts:
    generate_continuation(prompt, max_new_tokens=80)
    print("\n")

Testing LoRA model with sample prompts:

Context: Deep reinforcement learning has been widely adopted in robotic navigation.

Generated Continuation: For example, in the case of robotic navigation, it is possible to learn the direction of the robot using the motion of the robot (e.g., the robot will move in the direction of the robot). However, it is not possible to learn the direction of the robot using the motion of the robot.
In addition, there is a need to combine the motion of the robot and the motion
--------------------------------------------------------------------------------


Context: The key findings of this research are

Generated Continuation: as follows: (1) The mean age of the patients is 65.5 years, (2) the mean age of the patients is 81.7 years, and (3) the mean age of the patients is 80.8 years, which are similar to those in previous studies. (4) The mean age of the patients is 80.7 years, and (5) the mean age
--------------------------------------------------------

# GRADIO

In [7]:
# Gradio UI
def generate_response(prompt, max_tokens=100, temperature=0.7, top_p=0.9):
    """
    Wrapper function for Gradio interface.
    Calls generate_continuation without printing.
    """
    return generate_continuation(
        prompt, 
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        print_result=False
    )

# Create Gradio interface
interface = gr.Interface(
    fn=generate_response,
    inputs=[
        gr.Textbox(
            lines=3, 
            placeholder="Enter your academic prompt here...",
            label="Input Prompt"
        ),
        gr.Slider(
            minimum=20,
            maximum=200,
            value=100,
            step=10,
            label="Max Tokens",
            info="Maximum number of tokens to generate"
        ),
        gr.Slider(
            minimum=0.1,
            maximum=1.5,
            value=0.7,
            step=0.1,
            label="Temperature",
            info="Higher = more creative, Lower = more focused"
        ),
        gr.Slider(
            minimum=0.1,
            maximum=1.0,
            value=0.9,
            step=0.05,
            label="Top-p (Nucleus Sampling)",
            info="Controls diversity of output"
        )
    ],
    outputs=gr.Textbox(
        lines=5,
        label="Generated Continuation"
    ),
    title="🎓 Academic Writing LoRA Model",
    description="Generate formal academic text continuations using a fine-tuned GPT-2 model with LoRA adapters.",
    examples=[
        ["Deep reinforcement learning has been widely adopted in robotic navigation.", 100, 0.7, 0.9],
        ["The key findings of this research are", 80, 0.7, 0.9],
        ["In recent years, machine learning has demonstrated", 120, 0.8, 0.9],
        ["The methodology employed in this study involves", 100, 0.6, 0.9],
        ["These results suggest that", 90, 0.7, 0.9]
    ],
    theme=gr.themes.Soft(),
    allow_flagging="never"
)

# Launch the interface
interface.launch(share=False, debug=True)

c:\Users\jason\anaconda3\envs\OpenAI\Lib\site-packages\gradio\interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
# To run Gradio, just execute the cell above. No need for a separate command.